In [1]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
from sklearn.cluster import MeanShift
from sklearn import preprocessing, cross_validation
import pandas as pd
style.use('ggplot')

C:\Users\Aman Deep Singh\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_excel('http://pythonprogramming.net/static/downloads/machine-learning-data/titanic.xls')
# we use the pandas inbuilt method to copy the dataframe because
# original_df = df stores a 'pointer' to df in original_df. ie, modifying df modifies original_df too
original_df = pd.DataFrame.copy(df)
df.drop(['body', 'name'], axis=1, inplace=True)
df.convert_objects(convert_numeric=True)
df.head()

C:\Users\Aman Deep Singh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,female,29.0000,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,0.9167,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
original_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [7]:
df.fillna(0, inplace=True)

def handle_non_numerical_data(df):
    columns = df.columns.values
    
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    # creating dict that contains new id per unique string
                    text_digit_vals[unique] = x
                    x += 1
            
            # now we map to the new 'id' value to replace the string
            df[column] = list(map(convert_to_int, df[column]))
            
    return df

In [8]:
df = handle_non_numerical_data(df)
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,0,29.0000,0,0,744,211.3375,124,0,1,145
1,1,1,1,0.9167,1,2,509,151.5500,24,0,6,297
2,1,0,0,2.0000,1,2,509,151.5500,24,0,0,297
3,1,0,1,30.0000,1,2,509,151.5500,24,0,0,297
4,1,0,0,25.0000,1,2,509,151.5500,24,0,0,297


In [9]:
# Add or remove features just to see the impact they have
df.drop(['ticket', 'home.dest'], axis=1, inplace=True)

In [11]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

clf = MeanShift()
clf.fit(X)

MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [14]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_
original_df['cluster_group'] = np.nan
for i in range(len(X)):
    original_df['cluster_group'].iloc[i] = labels[i]
    
n_clusters_ = len(np.unique(labels))

survival_rates = {}
for i in range(n_clusters_):
    temp_df = original_df[(original_df['cluster_group'] == float(i))]
    survival_cluster = temp_df[(temp_df['survived'] == 1)]
    survival_rate = len(survival_cluster)/len(temp_df)
    survival_rates[i] = survival_rate
    
print(survival_rates)

C:\Users\Aman Deep Singh\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


{0: 0.36901865369018655, 1: 1.0, 2: 0.7291666666666666, 3: 0.1, 4: 0.0, 5: 1.0, 6: 1.0}


In [16]:
original_df[(original_df['cluster_group'] == 0)]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cluster_group
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",0.0
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY",0.0
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NaN,"Hudson, NY",0.0
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI",0.0
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY",0.0
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay",0.0
10,1,0,"Astor, Col. John Jacob",male,47.0000,1,0,PC 17757,227.5250,C62 C64,C,NaN,124.0,"New York, NY",0.0
12,1,1,"Aubart, Mme. Leontine Pauline",female,24.0000,0,0,PC 17477,69.3000,B35,C,9,NaN,"Paris, France",0.0
13,1,1,"Barber, Miss. Ellen ""Nellie""",female,26.0000,0,0,19877,78.8500,NaN,S,6,NaN,NaN,0.0
14,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0000,0,0,27042,30.0000,A23,S,B,NaN,"Hessle, Yorks",0.0


In [18]:
original_df[(original_df['cluster_group'] == 0)].describe()

,pclass,survived,age,sibsp,parch,fare,body,cluster_group
count,1233.000000,1233.000000,981.000000,1233.000000,1233.000000,1232.000000,115.000000,1233.0
mean,2.343877,0.369019,29.438498,0.429846,0.288727,24.586397,161.469565,0.0
std,0.810942,0.482735,14.258364,0.835583,0.640372,27.778030,98.326642,0.0
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000,0.0
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,71.000000,0.0
50%,3.000000,0.000000,28.000000,0.000000,0.000000,13.000000,165.000000,0.0
75%,3.000000,1.000000,38.000000,1.000000,0.000000,27.720800,257.000000,0.0
max,3.000000,1.000000,80.000000,5.000000,4.000000,227.525000,328.000000,0.0


In [19]:
original_df[(original_df['cluster_group'] == 2)]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cluster_group
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",2.0
2,1,0,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",2.0
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",2.0
11,1,1,"Astor, Mrs. John Jacob (Madeleine Talmadge Force)",female,18.0,1,0,PC 17757,227.5250,C62 C64,C,4,NaN,"New York, NY",2.0
17,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.0,0,1,PC 17558,247.5208,B58 B60,C,6,NaN,"Montreal, PQ",2.0
23,1,1,"Bidois, Miss. Rosalie",female,42.0,0,0,PC 17757,227.5250,NaN,C,4,NaN,NaN,2.0
24,1,1,"Bird, Miss. Ellen",female,29.0,0,0,PC 17483,221.7792,C97,S,8,NaN,NaN,2.0
35,1,1,"Bowen, Miss. Grace Scott",female,45.0,0,0,PC 17608,262.3750,NaN,C,4,NaN,"Cooperstown, NY",2.0
54,1,1,"Carter, Master. William Thornton II",male,11.0,1,2,113760,120.0000,B96 B98,S,4,NaN,"Bryn Mawr, PA",2.0
55,1,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,B96 B98,S,4,NaN,"Bryn Mawr, PA",2.0


In [20]:
original_df[(original_df['cluster_group'] == 2)].describe()

,pclass,survived,age,sibsp,parch,fare,body,cluster_group
count,48.000000,48.000000,39.000000,48.000000,48.000000,48.000000,1.0,48.0
mean,1.375000,0.729167,30.500000,2.229167,1.208333,168.819358,67.0,2.0
std,0.788886,0.449093,13.810275,2.926326,0.874176,72.062133,NaN,0.0
min,1.000000,0.000000,2.000000,0.000000,0.000000,52.000000,67.0,2.0
25%,1.000000,0.000000,20.000000,0.000000,0.000000,117.720825,67.0,2.0
50%,1.000000,1.000000,29.000000,1.000000,1.500000,159.164600,67.0,2.0
75%,1.000000,1.000000,40.000000,3.000000,2.000000,227.525000,67.0,2.0
max,3.000000,1.000000,63.000000,8.000000,2.000000,263.000000,67.0,2.0
